In [1]:
import os
import sys
import json
import torch

In [2]:
# inspect /bodmas_batches/train/batch_0139.pt 

file = '/data/saranyav/gcn_new/bodmas_batches/train/batch_0147.pt'


In [3]:
import torch
from torch_geometric.data import DataLoader
import json
# logger
import logging 
logger = logging.getLogger(__name__)
import os 
def load_batch(batch_file, family_to_group, batch_size=32):

    """Load and preprocess a single batch file."""
    try:
        if not os.path.exists(batch_file):
            logger.warning(f"Batch file not found: {batch_file}")
            return None
            
        # Load batch from file
        batch_data = torch.load(batch_file)
        processed = []
        
        for graph in batch_data:
            try:
                # Handle family and group labels
                family = getattr(graph, 'family', 'none')
                if not family or family == '':
                    family = 'none'
                    
                # Convert group to proper integer tensor
                group_id = family_to_group.get(family, -1)
                graph.group = torch.tensor([group_id], dtype=torch.long)
                
                # Edge handling
                if hasattr(graph, 'edge_attr') and graph.edge_attr.shape[0] != graph.edge_index.shape[1]:
                    # Fix edge attributes if dimensions don't match
                    num_edges = graph.edge_index.shape[1]
                    graph.edge_attr = torch.ones((num_edges, 1), dtype=torch.float)
                
                # Ensure types are correct
                graph.edge_index = graph.edge_index.long()
                if hasattr(graph, 'edge_attr'):
                    graph.edge_attr = graph.edge_attr.float()
                graph.x = graph.x.float()
                
                processed.append(graph)
                
            except Exception as e:
                logger.error(f"Error processing graph in {batch_file}: {str(e)}")
                continue
                
        if not processed:
            logger.warning(f"No valid graphs found in {batch_file}")
            return None
            
        # Create DataLoader
        try:
            return DataLoader(
                processed, 
                batch_size=min(batch_size, len(processed)),
                shuffle=False
            )
        except Exception as e:
            logger.error(f"Error creating DataLoader: {str(e)}")
            return None
        
    except Exception as e:
        logger.error(f"Error loading batch {batch_file}: {str(e)}")
        return None
    

# run 
family_to_group = json.load(open('/data/saranyav/gcn_new/behavioral_analysis/behavioral_groups.json'))
batch = load_batch(file, family_to_group, batch_size=32)


/tmp/ipykernel_159806/1885438647.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  batch_data = torch.load(batch_file)
/home/saranyav/.conda/envs/ctf/lib/python3.11/site-

In [14]:
def prepare_group_mappings(json_path):
    """Convert JSON group mappings to use integer group IDs."""
    with open(json_path, 'r') as f:
        raw_data = json.load(f)
    
    # Convert string group IDs to integers
    family_to_group = {}
    for group_id_str, families in raw_data.items():
        group_id = int(group_id_str)
        for family in families:
            family_to_group[family] = group_id
            
    return family_to_group

# Example usage
json_path = '/data/saranyav/gcn_new/behavioral_analysis/behavioral_groups.json'
family_to_group = prepare_group_mappings(json_path)
file_path = "/data/saranyav/gcn_new/bodmas_batches/train/batch_0147.pt"
batch = load_batch(file_path, family_to_group, batch_size=32)

/tmp/ipykernel_3943871/1885438647.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  batch_data = torch.load(batch_file)
/home/saranyav/.conda/envs/ctf/lib/python3.11/site

In [16]:
family_to_group

{'dapato': 2,
 'coinminer': 2,
 'grandcrab': 2,
 'upatre': 2,
 'unruy': 2,
 'plite': 2,
 'autorun': 2,
 'urelas': 2,
 'small': 2,
 'padodor': 2,
 'fuery': 2,
 'trickbot': 2,
 'wacatac': 2,
 'skeeyah': 2,
 'lunam': 2,
 'gc13003b': 2,
 'vflooder': 2,
 'gupboot': 2,
 'omaneat': 2,
 'fuerboos': 2,
 'skeeeyah': 2,
 'qqpass': 2,
 'ceeinject': 2,
 'autoit': 2,
 'gepys': 2,
 'tescrypt': 2,
 'blocker': 2,
 'vbclone': 2,
 'nanocore': 2,
 'dorv': 2,
 'tinba': 2,
 'fukru': 2,
 'glupteba': 2,
 'shifu': 2,
 'delf': 2,
 'ditertag': 2,
 'aenjaris': 2,
 'bladabindi': 2,
 'pkeylog': 2,
 'nanobot': 2,
 'emelent': 2,
 'neconyd': 2,
 'injector': 2,
 'kirts': 2,
 'bho': 2,
 'dynamer': 2,
 'smokeloader': 2,
 'tiggre': 2,
 'trickybot': 2,
 'revetrat': 2,
 'nitol': 2,
 'banker': 2,
 'systex': 2,
 'noancooe': 2,
 'mocrt': 2,
 'sillyfdc': 2,
 'shipup': 2,
 'sakurel': 2,
 'simbot': 2,
 'ursnif': 2,
 'autinject': 2,
 'plugx': 2,
 'occamy': 2,
 'quasarrat': 2,
 'Unknown': 2,
 'vbinject': 2,
 'ircbot': 2,
 'cryptomi

In [6]:
# # check if each item in this file has edge_attr
# # Example content:
# # Data(x=[6420, 14], edge_index=[2, 6655], edge_attr=[2072, 1], num_nodes=6420, sha='24a686ae67f97fbe479bb4c49f1ee2b6c7bd5bd7c0b6d59767dd14d36da14ca2', timestamp='2019-08-31 22:39:01 UTC', family='vflooder', malware_type='trojan')

# print("\nChecking for edge_attr in each item:")

# if isinstance(data, dict):
#     for key, value in data.items():
#         if hasattr(value, 'edge_attr'):
#             print(f"Key: {key}, edge_attr shape: {value.edge_attr.shape}")
#         else:
#             print(f"Key: {key}, edge_attr not found")


Checking for edge_attr in each item:


In [9]:
batch_data = torch.load('/data/saranyav/gcn_new/bodmas_batches/train/batch_0157.pt')
# for i, graph in enumerate(batch_data):
#     print(f"Graph {i}:")
#     print(f"x shape: {getattr(graph, 'x', None)}")
#     print(f"edge_index shape: {getattr(graph, 'edge_index', None)}")
#     print(f"edge_attr shape: {getattr(graph, 'edge_attr', None)}")


# Inspect the batch content
for i, graph in enumerate(batch):
    print(f"Graph {i}:")
    print(f"  x shape: {graph.x.shape}")
    print(f"  edge_index shape: {graph.edge_index.shape}")
    print(f"  edge_attr shape: {graph.edge_attr.shape if graph.edge_attr is not None else 'None'}")
    print(f"  Family: {getattr(graph, 'family', None)}")
    print(f"  y: {getattr(graph, 'y', None)}")
    print(f"  Is y None? {'Yes' if getattr(graph, 'y', None) is None else 'No'}")
    print("-" * 50)

Graph 0:
x shape: tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]])
edge_index shape: tensor([[  0,   1,   3,   6,   6,   7,   7,   8,   9,   9,  10,  11,  11,  12,
          13,  13,  15,  15,  16,  17,  18,  18,  19,  20,  20,  21,  22,  23,
          24,  25,  26,  26,  27,  28,  28,  29,  29,  30,  31,  31,  32,  33,
          33,  34,  35,  36,  37,  38,  39,  42,  44,  45,  47,  48,  49,  50,
          50,  51,  52,  52,  53,  54,  55,  56,  57,  58,  60,  60,  61,  62,
          63,  64,  64,  65,  66,  66,  67,  68,  69,  70,  70,  71,  71,  72,
          73,  74,  77,  79,  80,  80,  81,  81,  82,  83,  84,  84,  85,  86,
          88,  89,  92,  92,  93,  93,  94,  94,  95,  96,  96,  97,  98,  99,
         100, 101, 101, 102, 103, 104, 104, 105, 106, 107, 107, 108, 108, 110,

/tmp/ipykernel_197774/2816107008.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  batch_data = torch.load('/data/saranyav/gcn_new/bodmas_batches/train/batch_0157.pt')


In [10]:
import torch



# Function to check for family='none'
def check_family_none(data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict) or isinstance(value, list):
                # Recursively inspect nested dictionaries or lists
                result = check_family_none(value)
                if result:
                    return result
            elif hasattr(value, 'family') and getattr(value, 'family', None) == 'none':
                return True
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, dict) or isinstance(item, list):
                # Recursively inspect nested lists or dictionaries
                result = check_family_none(item)
                if result:
                    return result
            elif hasattr(item, 'family') and getattr(item, 'family', None) == 'none':
                return True
    elif hasattr(data, 'family') and getattr(data, 'family', None) == 'none':
        return True
    return False

# Check for family='none'
found = check_family_none(data)
if found:
    print("A key or attribute with family='none' exists in the data.")
else:
    print("No key or attribute with family='none' was found in the data.")

No key or attribute with family='none' was found in the data.


In [8]:
import glob
from pathlib import Path

batch_files = glob.glob('/data/saranyav/gcn_new/bodmas_batches/train/batch_*.pt')

for batch_file in batch_files:
    #print(f"Inspecting {batch_file}...")
    batch = torch.load(batch_file)

    for i, graph in enumerate(batch):
        #print(f"Graph {i}: edge_attr exists? {'edge_attr' in graph}")
        if not hasattr(graph, 'edge_attr') or graph.edge_attr is None:
            print(f"Graph {i} is missing edge_attr. Assigning default value.")

/tmp/ipykernel_197774/3341132874.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  batch = torch.load(batch_file)


In [ ]:
import os
import torch

# Directory containing the .pt files
directory =  "/data/saranyav/gcn_new/bodmas_batches/train"


# Function to check if family='none' exists in a data object
def has_family_none(data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                if has_family_none(value):
                    return True
            elif hasattr(value, 'family') and getattr(value, 'family', None) == 'none':
                return True
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                if has_family_none(item):
                    return True
            elif hasattr(item, 'family') and getattr(item, 'family', None) == 'none':
                return True
    elif hasattr(data, 'family') and getattr(data, 'family', None) == 'none':
        return True
    return False

# Iterate through all .pt files in the directory
for root, _, files in os.walk(directory):
    for file in files:
        if file.endswith(".pt"):
            file_path = os.path.join(root, file)
            try:
                # Load the .pt file
                data = torch.load(file_path)
                # Check for family='none'
                if has_family_none(data):
                    print(f"Found family='none' in file: {file_path}")
            except Exception as e:
                print(f"Could not process file {file_path}: {e}")